<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Validating-method-output" data-toc-modified-id="Validating-method-output-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Validating method output</a></span></li></ul></div>

In [3]:
import CRPS.CRPS as score  # pip package for validating output

In [5]:
import numpy as np
import torch
from torch.distributions.normal import Normal as N

$CRPS(\mathcal{N}(\mu, \sigma), x) = \sigma \bigl[ \frac{1}{\sqrt{\pi}} - 2 \phi(\frac{x-\mu}{\sigma}) - \frac{x-\mu}{\sigma}\bigl(2\Phi(\frac{x-\mu}{\sigma}) -1 \bigr)\bigr]$

In [36]:
def crps_gaussian(mu, sigma, x):
    '''Function to compute the Continuous Ranked Probability Score (CRPS) of an observed value, using a Gaussian
    distribution with mean mu and variance sigma^2 as a reference. 
    
    Parameters
    ----------
    mu : float
        The mean of the reference Gaussian distribution
    sigma : float
        The standard deviation of the reference Gaussian distribution
    x : float
        The observed value to score
    
    Returns
    -------
    crps : torch.tensor
        The CRPS of the observed value
    '''
    
    # Convert float values to tensors for use in PyTorch functions
    mu, sigma, x = torch.tensor([mu]), torch.tensor([sigma]), torch.tensor([x])
    
    # Compute shorthands of the normalized observation and pi, and create a standard normal distribution
    norm = (x-mu) / sigma
    Z = N(loc=0, scale=1)
    pi = torch.tensor([torch.acos(torch.zeros(1)).item() * 2])
    
    # Compute CRPS
    crps = sigma * (norm*(2*Z.cdf(norm) - 1) + 2*torch.exp(Z.log_prob(norm)) - 1/torch.sqrt(pi))
    return crps 

In [37]:
crps_gaussian(0.1, 0.01, 1)

tensor([0.8944])

In [38]:
crps_gaussian(0.1, 0.01, 0.1)

tensor([0.0023])

In [39]:
crps_gaussian(0.01, 1, 0.1)

tensor([0.2369])

## Validating method output

In [26]:
ens1 = np.random.normal(loc=0.1, scale=0.01, size=100)
score(ens1, 1).compute()

(0.8940276811294227, 0.8939691962819243, 0.8939984387056734)

In [20]:
score(ens1, 0.1).compute()

(0.002804501683248175, 0.0027397814499601036, 0.002772141566604139)

In [21]:
ens2 = np.random.normal(loc=0.01, scale=1, size=100)
score(ens2, 0.1).compute()

(0.23734815897161496, 0.23170889256698543, 0.2345285257693002)